In [74]:
import pandas as pd
from pandasql import sqldf
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as scs
pysqldf = lambda q: sqldf(q, globals())
import random
import statsmodels.api as sm

# Appendix

In [4]:
data = pd.read_csv("https://raw.githubusercontent.com/wheremagichappens/an.dy/master/samples/applicant_data.csv")

In [5]:
# For some reason column = group does not work in pandasql so had to change to grouping.
data.rename(columns={'group':'grouping'}, inplace=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108328 entries, 0 to 108327
Data columns (total 6 columns):
applicant_id    108328 non-null int64
channel         108328 non-null object
grouping        108328 non-null object
city            108328 non-null object
event           108328 non-null object
event_date      108328 non-null object
dtypes: int64(1), object(5)
memory usage: 2.9+ MB


In [7]:
# Group the data 
grouped_data = data.groupby(by = ['grouping', 'event'])

# Aggregate the data
data_summary = grouped_data.applicant_id.count()

# Examine the results
print()

In [8]:
list(data['event'].drop_duplicates())

['application_date',
 'card_mailed_date',
 'orientation_completed_date',
 'card_activation_date',
 'background_check_initiated_date',
 'background_check_completed_date',
 'first_batch_completed_date']

In [9]:
# control group (non-randomized) - By event
for i in list(data['event'].drop_duplicates()):
    print(i, round(data_summary['control'][i] / data_summary['control']['application_date'] * 100, 1))

application_date 100.0
card_mailed_date 98.4
orientation_completed_date 41.3
card_activation_date 74.7
background_check_initiated_date 74.7
background_check_completed_date 65.2
first_batch_completed_date 19.8


In [10]:
# treatment group - By event
for i in list(data['event'].drop_duplicates()):
    print(i, round(data_summary['treatment'][i] / data_summary['treatment']['application_date'] * 100, 1))

application_date 100.0
card_mailed_date 98.2
orientation_completed_date 46.8
card_activation_date 81.8
background_check_initiated_date 100.0
background_check_completed_date 88.7
first_batch_completed_date 34.3


In [11]:
# number of applicant_ids with first_batch_completed for control group (non-randomized)
c_converted = pysqldf("SELECT count(distinct applicant_id) as cnt_fbc FROM data WHERE  grouping = 'control' and event = 'first_batch_completed_date'")
c_converted = c_converted['cnt_fbc'][0]
c_converted

2873

In [12]:
# number of total applicant_ids for control group (non-randomized)
c_total = pysqldf("SELECT count(distinct applicant_id) as cnt_total FROM data WHERE  grouping = 'control'")
c_total = c_total['cnt_total'][0]
c_total

14501

In [13]:
# number of applicant_ids with first_batch_completed for treatment group
t_converted = pysqldf("SELECT count(distinct applicant_id) as cnt_fbc FROM data WHERE  grouping = 'treatment' and event = 'first_batch_completed_date'")
t_converted = t_converted['cnt_fbc'][0]
t_converted

2471

In [14]:
# number of total applicant_ids for treatment group
t_total = pysqldf("SELECT count(distinct applicant_id) as cnt_total FROM data WHERE  grouping = 'treatment'")
t_total = t_total['cnt_total'][0]
t_total

7197

In [15]:
# Get distinct applicant_id from grouping = 'control' and randomly select 7197 ids
c_distinct_id = pysqldf("SELECT distinct applicant_id FROM data WHERE grouping = 'control'")
c_distinct_id_list = list(c_distinct_id['applicant_id'])

In [16]:
# since we have unbalanced control vs treatment group sample size, we will do randomization to balance sample size (50 / 50)
# we will use 7197 applicant_ids for both control and treatment groups
random_id = random.sample(c_distinct_id_list, t_total)
c_total_random = data[data.applicant_id.isin(random_id)]

In [17]:
# let's get converted count from control group after randomization.
c_converted_random = pysqldf("select count(distinct applicant_id) as cnt_fbc from c_total_random where event = 'first_batch_completed_date'")
c_converted_random = c_converted_random['cnt_fbc'][0]
c_converted_random

1402

In [18]:
# Now, we know total sample size of control group (randomized) is 7197 (50 / 50).
c_total_random = len(c_total_random.applicant_id.drop_duplicates())
c_total_random

7197

In [19]:
# baseline conversion rate (non-randomized - float) = control conversion rate: ( total number of first_batch_completed in control (non-randomized)  / total number of applicant_id in control (non-randomized) )
c_conversion = (c_converted/c_total) 
c_conversion
# bcr = c_conversion
#bcr

0.1981242672919109

In [20]:
# baseline conversion rate (randomized - float) = control conversion rate: ( total number of first_batch_completed in control (randomized)  / total number of applicant_id in control (randomized) )
c_conversion_random = (c_converted_random/c_total_random)
bcr = c_conversion_random
bcr

0.19480339030151453

In [21]:
# Make a dataframe that only contains control (randomized) information
control_random_data = data[data.applicant_id.isin(random_id)]

In [22]:
# Group the data 
grouped_data = control_random_data.groupby(by = ['event'])

# Aggregate the data
data_summary = grouped_data.applicant_id.count()

# Examine the results
print(data_summary)

event
application_date                   7197
background_check_completed_date    4714
background_check_initiated_date    5368
card_activation_date               5368
card_mailed_date                   7088
first_batch_completed_date         1402
orientation_completed_date         2977
Name: applicant_id, dtype: int64


In [23]:
# control group (randomized) - By event 
for i in list(data['event'].drop_duplicates()):
    print(i, round(data_summary[i] / data_summary['application_date'] * 100, 1))

application_date 100.0
card_mailed_date 98.5
orientation_completed_date 41.4
card_activation_date 74.6
background_check_initiated_date 74.6
background_check_completed_date 65.5
first_batch_completed_date 19.5


In [24]:
# treatment conversion rate (float): ( total number of first_batch_completed in treatment  / total number of applicant_id in treatment )
t_conversion = (t_converted/t_total) 
t_conversion

0.3433375017368348

In [25]:
# mde (float): minimum change in measurement between control group (randomized) and test group if alternative hypothesis is true, sometimes referred to as minimum detectable effect
d_hat = t_conversion - c_conversion_random
mde = t_conversion - c_conversion_random
mde

0.14853411143532028

In [26]:
# From online calculator, https://www.evanmiller.org/ab-testing/sample-size.html, we know that our sampling size is reasonable (>126) for both control and treatment group

In [27]:
# Let's perform proportion test to see if our hypothesis is correct.

# one-sided (larger) & two-sided

# H0: conversion rate difference between treatment and control group is 0.
# Ha: conversion rate difference between treatment and control group is not 0.
count = np.array([t_converted, c_converted_random])
nobs = np.array([t_total, c_total_random])

In [28]:
# Find z-score and p-value from proportion test.
# From the result, we know p-value < 0.05. We reject null hypothesis (statistically significant).
# the difference between conversion rate in control and treatment is not 0. 
# our d_hat = mde was 14.85%. This improvement in conversion rate after changing the hiring process is not due to random chance.
# we are more than 99% confident to reject null hypothesis as p-value is even less than 0.01 for both two-sided and larger case.

#two-sided (two-sided, Ha: prop_t != prop_c)
z_score, p_value = sm.stats.proportions_ztest(count = count ,nobs = nobs, alternative = 'two-sided')
print("z-score:", z_score,
     "\np-value:", p_value)

#one-sided (larger, Ha: prop_t > prop_c)
z_score, p_value = sm.stats.proportions_ztest(count = count ,nobs = nobs, alternative = 'larger')
print("z-score:", z_score,
     "\np-value:", p_value)

#one-sided (Smaller, Ha: prop_t < prop_c)
z_score, p_value = sm.stats.proportions_ztest(count = count ,nobs = nobs, alternative = 'smaller')
print("z-score:", z_score,
     "\np-value:", p_value)

z-score: 20.091677365447357 
p-value: 8.726371001058168e-90
z-score: 20.091677365447357 
p-value: 4.363185500529084e-90
z-score: 20.091677365447357 
p-value: 1.0


In [29]:
# total cost would be paying background check (initial check)
# We need to pay $30 for each applicants no matter what
# How do we determine sample size for total cost? well, you can apply it to population but this time, let's apply it to control (non-randomized) for simplicity
# We will use control group (non-randomized - early background check) as sample to determine total cost
# Note that diff_converted can vary depending on the sample size you choose
# If we apply this logic to pouplation, c_total should be replaced with population_total
diff_converted = (t_conversion * c_total) - (c_conversion * c_total)  # total # of converted after the change in control (non-randomized) – total # of converted before the change in control (non-randomized)
total_cost = 30 * (diff_converted)
total_cost

63172.11338057525

In [55]:
# Merge control_random_data and treatment_data
control_random_treatment_merged = pysqldf("SELECT * FROM control_random_data UNION select * from data WHERE grouping = 'treatment'")

In [56]:
# Group the data 
grouped_data = control_random_treatment_merged.groupby(by = ['grouping', 'channel'])

# Aggregate the data
data_summary = grouped_data.applicant_id.nunique()

# Examine the results
print(data_summary)

grouping   channel               
control    job-search-site           1209
           shopper-referral-bonus    1253
           social-media              2036
           web-search-engine         2699
treatment  job-search-site           1187
           shopper-referral-bonus    1233
           social-media              2000
           web-search-engine         2777
Name: applicant_id, dtype: int64


In [57]:
# treatment group - channel ratio
data_summary['treatment'] / 7197

channel
job-search-site           0.164930
shopper-referral-bonus    0.171321
social-media              0.277894
web-search-engine         0.385855
Name: applicant_id, dtype: float64

In [59]:
# control (randomized) group - channel ratio
data_summary['control'] / 7197

channel
job-search-site           0.167987
shopper-referral-bonus    0.174100
social-media              0.282896
web-search-engine         0.375017
Name: applicant_id, dtype: float64

In [ ]:
# From above, we know that application ratio using web-search-engine is 1% higher in treatment when the ratio for every other channel is higher in control (randomized)
# Could higher ratio in web-search-engine in treatment group affect difference in conversion rate (control vs treatment)?
# Do shoppers who applied through web-search-engine have higher chance to complete batch (control vs treatment)?

In [68]:
# Group the data 
grouped_data = control_random_treatment_merged.groupby(by = ['grouping', 'channel', 'event'])

# Aggregate the data
data_summary = grouped_data.applicant_id.nunique()

# Examine the results
print(data_summary)

grouping   channel                 event                          
control    job-search-site         application_date                   1209
                                   background_check_completed_date     677
                                   background_check_initiated_date     760
                                   card_activation_date                760
                                   card_mailed_date                   1196
                                   first_batch_completed_date          149
                                   orientation_completed_date          593
           shopper-referral-bonus  application_date                   1253
                                   background_check_completed_date     796
                                   background_check_initiated_date     978
                                   card_activation_date                978
                                   card_mailed_date                   1223
                                 

In [78]:
# Group the data 
grouped_data = control_random_treatment_merged.groupby(by = ['grouping', 'city', 'event'])

# Aggregate the data
data_summary = grouped_data.applicant_id.nunique()

# Examine the results
print(data_summary)

grouping   city          event                          
control    Alfheim       application_date                    650
                         background_check_completed_date     417
                         background_check_initiated_date     470
                         card_activation_date                470
                         card_mailed_date                    633
                         first_batch_completed_date          138
                         orientation_completed_date          257
           Asgard        application_date                   2286
                         background_check_completed_date    1495
                         background_check_initiated_date    1703
                         card_activation_date               1703
                         card_mailed_date                   2254
                         first_batch_completed_date          438
                         orientation_completed_date          927
           Helheim       applicat

In [79]:
# treatment group - by city (batch completed ratio) 
for i in list(data['city'].drop_duplicates()):
    print(i, round(data_summary['treatment'][i]['first_batch_completed_date'] / data_summary['treatment'][i]['application_date'] * 100,1))

Asgard 34.2
Midgard 34.7
Helheim 23.6
Alfheim 32.4
Jotunheim 36.9
Niflheim 38.6
Vanaheim 34.5
Muspelheim 27.6
Svartalfheim 31.9


In [80]:
# control (randomized) group - by city (batch completed ratio) 
for i in list(data['city'].drop_duplicates()):
    print(i, round(data_summary['control'][i]['first_batch_completed_date'] / data_summary['treatment'][i]['application_date'] * 100,1))

Asgard 19.0
Midgard 19.1
Helheim 16.4
Alfheim 22.5
Jotunheim 25.2
Niflheim 15.3
Vanaheim 22.8
Muspelheim 12.1
Svartalfheim 23.4


In [86]:
# treatment group - by city
for i in list(data['city'].drop_duplicates()):
    print(i, data_summary['treatment'][i] / data_summary['treatment'][i]['application_date'])

Asgard event
application_date                   1.000000
background_check_completed_date    0.891587
background_check_initiated_date    1.000000
card_activation_date               0.816565
card_mailed_date                   0.983088
first_batch_completed_date         0.341717
orientation_completed_date         0.466609
Name: applicant_id, dtype: float64
Midgard event
application_date                   1.000000
background_check_completed_date    0.883749
background_check_initiated_date    1.000000
card_activation_date               0.812538
card_mailed_date                   0.980523
first_batch_completed_date         0.346622
orientation_completed_date         0.468351
Name: applicant_id, dtype: float64
Helheim event
application_date                   1.000000
background_check_completed_date    0.890909
background_check_initiated_date    1.000000
card_activation_date               0.909091
card_mailed_date                   0.963636
first_batch_completed_date         0.236364
orientati

In [87]:
# control (randomized) group - by city
for i in list(data['city'].drop_duplicates()):
    print(i, data_summary['control'][i] / data_summary['control'][i]['application_date'])

Asgard event
application_date                   1.000000
background_check_completed_date    0.653981
background_check_initiated_date    0.744969
card_activation_date               0.744969
card_mailed_date                   0.986002
first_batch_completed_date         0.191601
orientation_completed_date         0.405512
Name: applicant_id, dtype: float64
Midgard event
application_date                   1.000000
background_check_completed_date    0.650076
background_check_initiated_date    0.744122
card_activation_date               0.744122
card_mailed_date                   0.985344
first_batch_completed_date         0.192061
orientation_completed_date         0.424122
Name: applicant_id, dtype: float64
Helheim event
application_date                   1.000000
background_check_completed_date    0.630435
background_check_initiated_date    0.804348
card_activation_date               0.804348
card_mailed_date                   1.000000
first_batch_completed_date         0.195652
orientati

In [71]:
# treatment group - by channel (batch completed ratio)
for i in list(data['channel'].drop_duplicates()):
    print(i, round(data_summary['treatment'][i]['first_batch_completed_date'] / data_summary['treatment'][i]['application_date'] * 100,1))

web-search-engine 36.7
social-media 31.5
job-search-site 32.1
shopper-referral-bonus 35.8


In [72]:
# control group (randomized) - by channel (batch completed ratio)
for i in list(data['channel'].drop_duplicates()):
    print(i, round(data_summary['control'][i]['first_batch_completed_date'] / data_summary['control'][i]['application_date'] * 100,1))

web-search-engine 18.6
social-media 23.1
job-search-site 12.3
shopper-referral-bonus 22.3


In [ ]:
# Notice the difference
# From treatment group, the highest conversion rate comes from web-search-engine (36.7%) and the lowest from social-media (31.5%)
# From control (randomized), the highest conversion rate comes from social-media (23.1%) and the lowest from job-search-site (12.3%)
# We may want to post more ads on web-search-engine after the change and less on social media from now on
# We may want to A/B test for conversion rate difference by each channel just like we did for back ground check
# If the result of A/B test is statistically significant, we know the difference is not due to random chance

In [49]:
# treatment group - first batched/application_date by channel
# We will calculate proportion for success for treatment group by channel
for i in list(data['channel'].drop_duplicates()):
    print(i, data_summary['treatment'][i][['first_batch_completed_date','application_date']])

web-search-engine event
first_batch_completed_date    1018
application_date              2777
Name: applicant_id, dtype: int64
social-media event
first_batch_completed_date     630
application_date              2000
Name: applicant_id, dtype: int64
job-search-site event
first_batch_completed_date     381
application_date              1187
Name: applicant_id, dtype: int64
shopper-referral-bonus event
first_batch_completed_date     442
application_date              1233
Name: applicant_id, dtype: int64


In [50]:
# control group (non-randomized) - first batched/application_date by channel
# We will calculate proportion for success for control group (non-randomized) by channel
for i in list(data['channel'].drop_duplicates()):
    print(i, data_summary['control'][i][['first_batch_completed_date','application_date']])

web-search-engine event
first_batch_completed_date     503
application_date              2699
Name: applicant_id, dtype: int64
social-media event
first_batch_completed_date     471
application_date              2036
Name: applicant_id, dtype: int64
job-search-site event
first_batch_completed_date     149
application_date              1209
Name: applicant_id, dtype: int64
shopper-referral-bonus event
first_batch_completed_date     279
application_date              1253
Name: applicant_id, dtype: int64


In [51]:
#two-sided (two-sided, Ha: prop_t != prop_c) - shopper-referral-bonus

z_score, p_value = sm.stats.proportions_ztest(count = [442, 279] ,nobs = [1233, 1253], alternative = 'two-sided')
print("z-score:", z_score,
     "\np-value:", p_value)

#one-sided (larger, Ha: prop_t > prop_c) - shopper-referral-bonus

z_score, p_value = sm.stats.proportions_ztest(count = [442, 279] ,nobs = [1233, 1253], alternative = 'larger')
print("z-score:", z_score,
     "\np-value:", p_value)


#onesided (smaller, Ha: prop_t < prop_c) - shopper-referral-bonus

z_score, p_value = sm.stats.proportions_ztest(count = [442, 279] ,nobs = [1233, 1253], alternative = 'smaller')
print("z-score:", z_score,
     "\np-value:", p_value)


z-score: 7.461021886680081 
p-value: 8.585398689090712e-14
z-score: 7.461021886680081 
p-value: 4.292699344545356e-14
z-score: 7.461021886680081 
p-value: 0.999999999999957


In [52]:
#two-sided (two-sided, Ha: prop_t != prop_c) - web-search-engine
z_score, p_value = sm.stats.proportions_ztest(count = [1018, 503] ,nobs = [2777, 2699], alternative = 'two-sided')
print("z-score:", z_score,
     "\np-value:", p_value)

#one-sided (larger, Ha: prop_t > prop_c) - web-search-engine
z_score, p_value = sm.stats.proportions_ztest(count = [1018, 503] ,nobs = [2777, 2699], alternative = 'larger')
print("z-score:", z_score,
     "\np-value:", p_value)


#one-sided (smaller, Ha: prop_t < prop_c) - web-search-engine
z_score, p_value = sm.stats.proportions_ztest(count = [1018, 503] ,nobs = [2777, 2699], alternative = 'smaller')
print("z-score:", z_score,
     "\np-value:", p_value)


z-score: 14.886060659219108 
p-value: 4.059964159161729e-50
z-score: 14.886060659219108 
p-value: 2.0299820795808646e-50
z-score: 14.886060659219108 
p-value: 1.0


In [53]:
#two-sided (two-sided, Ha: prop_t != prop_c) - job-search-site
z_score, p_value = sm.stats.proportions_ztest(count = [381, 149] ,nobs = [1187, 1209], alternative = 'two-sided')
print("z-score:", z_score,
     "\np-value:", p_value)

#one-sided (larger, Ha: prop_t > prop_c) - job-search-site
z_score, p_value = sm.stats.proportions_ztest(count = [381, 149] ,nobs = [1187, 1209], alternative = 'larger')
print("z-score:", z_score,
     "\np-value:", p_value)


#one-sided (smaller, Ha: prop_t < prop_c) - job-search-site
z_score, p_value = sm.stats.proportions_ztest(count = [381, 149] ,nobs = [1187, 1209], alternative = 'smaller')
print("z-score:", z_score,
     "\np-value:", p_value)


z-score: 11.659273790704658 
p-value: 2.0579005859118713e-31
z-score: 11.659273790704658 
p-value: 1.0289502929559356e-31
z-score: 11.659273790704658 
p-value: 1.0


In [54]:
#two-sided (two-sided, Ha: prop_t != prop_c) - social-media
z_score, p_value = sm.stats.proportions_ztest(count = [630, 471] ,nobs = [2000, 2036], alternative = 'two-sided')
print("z-score:", z_score,
     "\np-value:", p_value)

#one-sided (larger, Ha: prop_t > prop_c) - social-media
z_score, p_value = sm.stats.proportions_ztest(count = [630, 471] ,nobs = [2000, 2036], alternative = 'larger')
print("z-score:", z_score,
     "\np-value:", p_value)


#one-sided (smaller, Ha: prop_t < prop_c) - social-media
z_score, p_value = sm.stats.proportions_ztest(count = [630, 471] ,nobs = [2000, 2036], alternative = 'smaller')
print("z-score:", z_score,
     "\np-value:", p_value)


z-score: 5.9665134011648995 
p-value: 2.423768404048761e-09
z-score: 5.9665134011648995 
p-value: 1.2118842020243805e-09
z-score: 5.9665134011648995 
p-value: 0.9999999987881159


In [ ]:
# first_batch_completed rate varries by each channel; it is worth it to examine further by channel for both control and treatment group.
# We may want to test proportion test for success rate (first_batch_completed_date) by channel.
# It could be that the significant difference in terms of success proportions between, say web-search-engine for control and treatment group, could have occurred due to random chance.
# What we tested early on was simply to figure out if OVERALL difference in terms of success proportions between control and treatment group was statistically significant.
# Before we test and confirm with p-value, we never know if EACH CHANNEL difference in terms of success proprotions between control and treatmnet group is statistically significant.

# We also might want to know if advertising strategy by channel is different one another in treatment group compared to control group.

# From the results above, we concluded that the difference between treatment and control in terms of conversion rate by channel is not due to random chance.